In [2]:
run mysetup.py

Setup complete.


## Solving Optimization 

In [16]:
excel_path    = pathlib.Path.cwd() / "input" / "20250604_093243" / "input.xlsx"

sheets = pd.read_excel(excel_path, sheet_name=None)    # sheet_name=None → dict
df_pic  = sheets['df_pic']
df_init = sheets['df_init']
df_S    = sheets['df_S']
df_so   = sheets['df_so']
df_sc   = sheets['df_sc']
df_ph   = sheets['df_ph']
df_tb   = sheets['df_tb']
df_tbl  = sheets['df_tbl']

In [17]:
N_PIC =  df_pic["PIC_ID"].nunique()
N_HOUSE = df_ph["HOUSE_ID"].nunique()
# identify SET columns by the prefix "SET"
set_cols = [c for c in df_S.columns if c.startswith("SET")]
N_SETS  = len(set_cols)

PIC     = df_pic["PIC_ID"].tolist()
HOUSE   = sorted(df_ph["HOUSE_ID"].unique().tolist())
SETS    = sorted(set_cols)  # keeps numerical order

# ------------- possible group labels --------------------------
# columns already follow the naming convention
MO    = ['MO', 'NO_MO'] # criterion 1: mental observation MO/NO_MO
VIOLENCE    = ['MIN', 'MED', 'MAX'] # criterion 2: violence levels MIN/MED/MAX
TYPES = df_tb["TYPE_ID"].unique().tolist()  # criterion 3: type of behavior

CRITERION = MO + VIOLENCE

In [18]:
prob = LpProblem("Housing_Min_Move", LpMinimize)

a = LpVariable.dicts('a', (PIC, HOUSE), 0, 1, LpBinary)   # assignment
b = LpVariable.dicts('b', (HOUSE, TYPES), 0, 1, LpBinary) # house type
u = LpVariable.dicts('u', HOUSE, 0, 1, LpBinary)          # house used
m = LpVariable.dicts('m', PIC, 0, 1, LpBinary)            # moved?


cap      = df_ph.set_index('HOUSE_ID')['CAP'].to_dict()
init_h   = df_init.set_index('PIC_ID')['HOUSE_ID_INIT'].to_dict()
x_is     = {(r.PIC_ID,s):r[s] for _,r in df_S.iterrows() for s in SETS}
y_il     = {(r.PIC_ID,l):r[l] for _,r in df_pic.iterrows() for l in CRITERION}
w_tl     = {(r.TYPE_ID,l):r[l] for _,r in df_tbl.iterrows() for l in CRITERION}
T_of_h   = {h:df_tb[df_tb.HOUSE_ID==h]['TYPE_ID'].tolist() for h in HOUSE}

In [19]:
# (i) each PIC in exactly one house
for i in PIC:
    prob += lpSum(a[i][h] for h in HOUSE) == 1

# (ii) one type per house
for h in HOUSE:
    prob += lpSum(b[h][t] for t in T_of_h[h]) == 1
    prob += lpSum(b[h][t] for t in TYPES) == 1

# (iii) link u_h (if house is utilized )
for h in HOUSE:
    for i in PIC:
        prob += a[i][h] <= u[h]

# (iv) capacity
for h in HOUSE:
    prob += lpSum(a[i][h] for i in PIC) <= cap[h]

# (v) movement flag  m_i = 1 – a_{i,init(i)}
for i in PIC:
    prob += m[i] + a[i][init_h[i]] == 1

# (vi) separation-order
for _,row in df_so.iterrows():
    p1,p2 = row.PIC_ID1, row.PIC_ID2
    for h in HOUSE:
        prob += a[p1][h] + a[p2][h] <= 1

# (vii) SRG conflicts
for _,row in df_sc.iterrows():
    s1,s2 = row.SET1, row.SET2
    for h in HOUSE:
        prob += (
            lpSum(a[i][h]*x_is[i,s1] for i in PIC) +
            lpSum(a[i][h]*x_is[i,s2] for i in PIC) <= 1)


# (viii) type-based limits
for h in HOUSE:
    for l in CRITERION:
        lhs = lpSum(a[i][h]*y_il[i,l] for i in PIC)
        rhs = cap[h]*lpSum(b[h][t]*w_tl[t,l] for t in T_of_h[h]) # upper bound on this criterion of that assigned house type
        prob += lhs <= rhs
        

# objective = moves  + 0.1 × houses-used
prob += lpSum(m[i] for i in PIC) + 0.1*lpSum(u[h] for h in HOUSE)
prob.solve(PULP_CBC_CMD(msg=True, timeLimit= 450))

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Users/wasinmeesena/Library/Python/3.9/lib/python/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/v4/ygpx35493v50lqrtm282cqnm0000gn/T/b0bb06093d2746b69ba9300b76956812-pulp.mps -sec 450 -timeMode elapsed -branch -printingOptions all -solution /var/folders/v4/ygpx35493v50lqrtm282cqnm0000gn/T/b0bb06093d2746b69ba9300b76956812-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 95695 COLUMNS
At line 836278 RHS
At line 931969 BOUNDS
At line 1023690 ENDATA
Problem MODEL has 95690 rows, 91720 columns and 556052 elements
Coin0008I MODEL read with 0 errors
seconds was changed from 1e+100 to 450
Option for timeMode changed from cpu to elapsed
Continuous objective value is 183 - 0.17 seconds
Cgl0002I 13717 variables fixed
Cgl0003I 43 fixed, 0 tightened bounds, 6815 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 289 strengthened rows, 0 substitu

1

In [20]:
# ▸ which houses ended up in use?
houses_used = [h for h in HOUSE if u[h].value() > 0.5]

house_rows = []
for h in houses_used:
    # house type chosen by the solver (only one is =1)
    chosen_type = next(
        t for t in T_of_h[h]
        if b[h][t].value() > 0.5
    )

    # list every PIC assigned to that house
    pic_list = [i for i in PIC if a[i][h].value() > 0.5]

    house_rows.append(dict(
        HOUSE_ID     = h,
        HOUSE_TYPE   = chosen_type,
        N_PIC        = len(pic_list),
        PIC_LIST     = ", ".join(pic_list)        # comma-separated IDs
    ))

houses = pd.DataFrame(house_rows).sort_values("HOUSE_ID")


In [26]:
timestamp   = datetime.datetime.now(ZoneInfo("America/New_York")).strftime("%Y%m%d_%H%M%S")
base_dir    = pathlib.Path.cwd() / "output" / timestamp
base_dir.mkdir(parents=True, exist_ok=True)

In [27]:
moves = pd.DataFrame([
    dict(PIC_ID=i,
         HOUSE_ID_INIT = init_h[i],
         HOUSE_ID_NEW  = next(h for h in HOUSE if a[i][h].value()>0.5))
    for i in PIC
])
moves['MOVED_FLAG'] = (moves.HOUSE_ID_INIT != moves.HOUSE_ID_NEW).astype(int)

summary = pd.DataFrame([{
    'TOTAL_MOVES' : int(moves.MOVED_FLAG.sum()),
    'HOUSES_USED' : int(sum(u[h].value() for h in HOUSE))
}])


path_result = base_dir / 'output_moves.xlsx'

with pd.ExcelWriter(path_result, engine="xlsxwriter") as writer:
    moves.to_excel(writer,   sheet_name="Moves",   index=False)
    summary.to_excel(writer, sheet_name="Summary", index=False)
    houses.to_excel(writer,  sheet_name="Houses",  index=False)
